Acá importamos las librerias que son necesarias para el resto del código. 

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt

Invocamos los datos que hemos importado tales como las 'reservas'. 'reporte de ingreso año 2024' y el 'check-in' 

In [ ]:
df = pd.read_csv(r'C:\Users\tabo_\OneDrive\Desktop\absolute\reservas 27032024 27032025.csv', sep=';', skiprows=1, encoding='utf-8')
df2 = pd.read_csv(r'C:\Users\tabo_\OneDrive\Desktop\absolute\reporte de ingreso año 2024 a la fecha.csv', sep=';', encoding='latin1')
df3 = pd.read_excel(r'C:\Users\tabo_\OneDrive\Desktop\absolute\Check-in del 2024-03-27 al 2025-03-28 bookin.xls', sheet_name='Sheet1', header=0)

invocamos el DataFrame de 'df'

In [ ]:
# Elimina los valores nulos para evitar errores
genero = df['GÃ©nero'].dropna()

# Cuenta cuántos hay de cada género
conteo = genero.value_counts()

# Calcula el porcentaje
porcentaje = genero.value_counts(normalize=True) * 100

# Muestra resultados
print("Conteo por género:\n", conteo)
print("\nPorcentaje por género:\n", porcentaje)


Los datos faltantes (missing_data) que hay en cada columna, osea los datos que la gente no registra. 

In [ ]:
# ...existing code...

# Contar los datos faltantes en cada columna
missing_data = df.isnull().sum()
print(missing_data)

# ...existing code...

En este código, buscamos eliminar los datos que no necesitamos cuyos datos faltantes exceden de los 2000 y a su vez los eliminamos de las columnas del DataFrame. 

In [ ]:
# ...existing code...

# Contar los datos faltantes en cada columna
missing_data = df.isnull().sum()

# Definir el umbral de datos faltantes (más de 2000)
threshold = 2000

# Identificar las columnas que exceden el umbral
columns_to_drop = missing_data[missing_data > threshold].index

# Eliminar las columnas del DataFrame
df = df.drop(columns=columns_to_drop)

# Mostrar las columnas eliminadas
print(f"Columnas eliminadas: {list(columns_to_drop)}")

# ...existing code...

La información de los datos que se han llenado en cada columna de los 2467 que se han registrado (entries)

In [ ]:
df.info()

In [ ]:
df2

Mostrar la informacion del segundo Data Frame "info()"

In [ ]:
df2.info()

Contamos los datos que faltan en cada columna y lo mostramos. 

In [ ]:
# ...existing code...

# Contar los datos faltantes en cada columna
missing_data = df2.isnull().sum()
print(missing_data)

# ...existing code...

En este codigo se procede a eliminar las columnas de los datos faltantes que se exceden mas de 9000 y se muestra abajo la lista de las columnas eliminadas.

In [ ]:
# ...existing code...

# Contar los datos faltantes en cada columna de df2
missing_data_df2 = df2.isnull().sum()

# Definir el umbral de datos faltantes (más de 9000)
threshold_df2 = 9000

# Identificar las columnas que exceden el umbral
columns_to_drop_df2 = missing_data_df2[missing_data_df2 > threshold_df2].index

# Eliminar las columnas del DataFrame df2
df2 = df2.drop(columns=columns_to_drop_df2)

# Mostrar las columnas eliminadas
print(f"Columnas eliminadas de df2: {list(columns_to_drop_df2)}")

# ...existing code...

In [ ]:
df3

In [ ]:
df3.info()

In [ ]:
missing_data = df3.isnull().sum()
print(missing_data)

In [ ]:
# ...existing code...

# Contar los datos faltantes en cada columna de df3
missing_data_df3 = df3.isnull().sum()

# Definir el umbral de datos faltantes (más de 2000)
threshold_df3 = 2000

# Identificar las columnas que exceden el umbral
columns_to_drop_df3 = missing_data_df3[missing_data_df3 > threshold_df3].index

# Eliminar las columnas del DataFrame df3
df3 = df3.drop(columns=columns_to_drop_df3)

# Mostrar las columnas eliminadas
print(f"Columnas eliminadas de df3: {list(columns_to_drop_df3)}")

# ...existing code...

In [ ]:
# ...existing code...

# Contar la cantidad de registros por cada motivo del viaje
motivo_viaje_counts = df3['Motivo del viaje'].value_counts()
print(motivo_viaje_counts)

# ...existing code...

In [ ]:
# ...existing code...

# Eliminar la columna "Motivo del viaje" del DataFrame df3
df3 = df3.drop(columns=['Motivo del viaje'])

# Mostrar un mensaje confirmando la eliminación
print("La columna 'Motivo del viaje' ha sido eliminada.")

# ...existing code...

In [ ]:
# ...existing code...

# Eliminar la columna "Dispositivo" del DataFrame df3
df3 = df3.drop(columns=['Dispositivo'])

# Mostrar un mensaje confirmando la eliminación
print("La columna 'Dispositivo' ha sido eliminada.")

In [ ]:
df2

### union de los datos segun el cliente 



In [ ]:

import unidecode

# Función para limpiar nombres
def normalizar(nombre):
    if pd.isna(nombre):
        return ''
    return unidecode.unidecode(str(nombre).lower().strip())

# Normalizamos en todos los DF
df['nombre_cliente'] = df['Titular'].apply(normalizar)
df2['nombre_cliente'] = df2['Cliente'].apply(normalizar)
df3['nombre_cliente'] = df3['Nombre del cliente (o clientes)'].apply(normalizar)


In [ ]:
# Unión reservas + consumos (cliente interno)
df_completo = pd.merge(df, df2, on='nombre_cliente', how='outer', suffixes=('_reserva', '_consumo'))

# Unión con info externa
df_completo1 = pd.merge(df_completo, df3, on='nombre_cliente', how='outer', suffixes=('', '_externa'))


In [67]:


# Asegúrate de trabajar sobre una copia del df
df = df_completo1.copy()

# 1. Filtrar solo reservas efectivas (Estado == 'ok')
df_efectivas = df[df['Estado'].str.lower() == 'ok']

# 2. Convertir 'Ingreso' a datetime
df_efectivas['Ingreso'] = pd.to_datetime(df_efectivas['Ingreso'], errors='coerce')

# 3. Crear columna 'Mes'
df_efectivas['Mes'] = df_efectivas['Ingreso'].dt.to_period('M')

# 4. Asegurarnos de que 'Total alojamiento' esté en formato numérico
df_efectivas['Total alojamiento'] = pd.to_numeric(df_efectivas['Total alojamiento'], errors='coerce')

# 5. Agrupar por mes y sumar ingresos
ingresos_mensuales = df_efectivas.groupby('Mes')['Total alojamiento'].sum()

# 6. Calcular el promedio mensual
promedio_mensual = ingresos_mensuales.mean()

# 7. Mostrar resultado
print("✅ Ingreso mensual promedio por reservas efectivas: ${:,.2f}".format(promedio_mensual))


✅ Ingreso mensual promedio por reservas efectivas: $20,057,271.00


C:\Users\tabo_\AppData\Local\Temp\ipykernel_3116\4272671812.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_efectivas['Ingreso'] = pd.to_datetime(df_efectivas['Ingreso'], errors='coerce')
C:\Users\tabo_\AppData\Local\Temp\ipykernel_3116\4272671812.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_efectivas['Mes'] = df_efectivas['Ingreso'].dt.to_period('M')
C:\Users\tabo_\AppData\Local\Temp\ipykernel_3116\4272671812.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [ ]:
df_completo

In [ ]:


# Sacar clientes únicos de cada grupo
clientes_durmieron = set(df_completo[df_completo['Titular'].notna()]['nombre_cliente'].dropna().unique())
clientes_compraron = set(df_completo[df_completo['Producto'].notna()]['nombre_cliente'].dropna().unique())

# Clasificar los grupos
solo_durmieron = clientes_durmieron - clientes_compraron
solo_compraron = clientes_compraron - clientes_durmieron
ambos = clientes_durmieron & clientes_compraron

# Contar cuántos hay en cada grupo
cuentas = {
    'Solo durmieron': len(solo_durmieron),
    'Solo compraron': len(solo_compraron),
    'Durmieron y compraron': len(ambos)
}


In [ ]:
# Graficar
%matplotlib inline


plt.figure(figsize=(8, 6))
plt.bar(cuentas.keys(), cuentas.values(), color=['skyblue', 'salmon', 'lightgreen']) 
plt.title('Comparación de clientes por actividad en el hotel')
plt.ylabel('Cantidad de clientes')
plt.xlabel('Categoría de cliente')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
